In [4]:
import yfinance as yf
import pandas as pd

stock_symbol = 'AAPL'
start_date = '2021-01-01'
end_date = '2023-01-01'

AAPL = yf.download(stock_symbol, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [5]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScale

scaler = MinMaxScaler(feature_range=(-1, 1)) 

Processing Data

In [12]:
closing_prices = AAPL['Close']

raw_cp_dat = closing_prices.values

# Standardizing
raw_cp_dat = raw_cp_dat - raw_cp_dat.mean()
raw_cp_dat = raw_cp_dat/raw_cp_dat.std()

# Scaling
raw_cp_dat = MinMaxScaler(raw_cp_dat)


LSTM Model Def

In [5]:
class LSTM(nn.Module):
    def __init__(self, input_size = 1, hidden_layer_size = 32, num_layers = 2, output_size = 1):
        super(LSTM, self).__init__()
        self.input_dim = input_size
        self.hidden_dim = hidden_layer_size
        self.num_layers = num_layers
        self.output_dim = output_size

        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True, dropout=0)
        self.fc = nn.Linear(self.hidden_dim, self.output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :])
        return out

In [7]:
model = LSTM()

criterion = torch.nn.MSELoss(reduction='sum')

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
import time

num_epochs = 500
hist = np.zeros(num_epochs)
start_time = time.time()
lstm = []
error = 1
tol = 1e-3

for t in range(num_epochs):
    if (error > tol):
        y_train_pred = model(x_train)
        loss = criterion(y_train_pred, y_train_lstm)
        error = abs(loss.item())
        if (t%10 == 0): print("Epoch ", t, "MSE: %.3e" % loss.item())
        if loss.item()<tol:
            print("Epoch ", t, "MSE: %.3e" % loss.item())
            break
        hist[t] = loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    else:
        break